In [ ]:
!pip install transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 16.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 41.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.5 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.6.1
    Uninstalling fsspec-2024.6.1:
     

In [ ]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel, Trainer, TrainingArguments
from datasets import load_dataset

In [ ]:
import pandas as pd
from datasets import Dataset

# Load your CSV file
df = pd.read_csv('/content/dialogue_texts.csv')




In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12154 entries, 0 to 12153
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   dialogueID  12154 non-null  object
 1   text        12154 non-null  object
dtypes: object(2)
memory usage: 190.0+ KB


In [ ]:
# Assuming each row in 'text' is a turn in a conversation,
# pair each turn with the next one to create prompt-response pairs.
prompts = []
responses = []

for i in range(len(df) - 1):
    prompts.append(df.iloc[i]['text'])
    responses.append(df.iloc[i + 1]['text'])

In [ ]:

# Prepare the dataset
data = {
    'prompt': prompts,
    'response': responses
}

In [ ]:
 #Create a DataFrame from the pairs
df_pairs = pd.DataFrame(data)

# Save the pairs to a new CSV
df_pairs.to_csv('prompts_responses.csv', index=False)

# Load the pairs into a Dataset
dataset = Dataset.from_pandas(df_pairs)

## **Tokenize**

In [ ]:
from transformers import GPT2Tokenizer

# Load the GPT-2 tokenizer
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

# Add a padding token to the tokenizer
tokenizer.pad_token = tokenizer.eos_token

def tokenize_function(examples):
    inputs = tokenizer(examples['prompt'], truncation=True, padding='max_length', max_length=512)
    outputs = tokenizer(examples['response'], truncation=True, padding='max_length', max_length=512)
    return {
        'input_ids': inputs['input_ids'],
        'attention_mask': inputs['attention_mask'],
        'labels': outputs['input_ids']
    }

tokenized_datasets = dataset.map(tokenize_function, batched=True)

Map:   0%|          | 0/12153 [00:00<?, ? examples/s]

In [ ]:
from transformers import GPT2LMHeadModel, Trainer, TrainingArguments

# Load pre-trained GPT-2 model
model = GPT2LMHeadModel.from_pretrained('gpt2')

# Define training arguments
training_args = TrainingArguments(
    output_dir="./gpt2-finetuned",
    per_device_train_batch_size=2,
    num_train_epochs=3,
    logging_dir="./logs",
    save_steps=10_000,
    save_total_limit=2,
)

# Initialize Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
)

# Train the model
trainer.train()

# Save the fine-tuned model and tokenizer
model.save_pretrained('fine_tuned_gpt2')
tokenizer.save_pretrained('fine_tuned_gpt2')


model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Step,Training Loss
500,0.305800
1000,0.217900
1500,0.200300
2000,0.205900
2500,0.190400
3000,0.195900
3500,0.207300
4000,0.193000
4500,0.200400
5000,0.196800


('fine_tuned_gpt2/tokenizer_config.json',
 'fine_tuned_gpt2/special_tokens_map.json',
 'fine_tuned_gpt2/vocab.json',
 'fine_tuned_gpt2/merges.txt',
 'fine_tuned_gpt2/added_tokens.json')